<a href="https://colab.research.google.com/github/PabloCGarcia/bioDeepLearning/blob/main/Clase10_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clase de introducción de deepchem, experimentos con deep learning con datasets conocidos

In [1]:
#!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
#import conda_installer
#conda_installer.install()
#!/root/miniconda/bin/conda info -e
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:32
🔁 Restarting kernel...


In [1]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [2]:
!conda install -y -c rdkit rdkit==2020.09.2

Solving environment: \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | /

In [3]:
!conda install -y -c conda-forge openmm
!conda install -y -c omnia pdbfixer

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - openmm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cudatoolkit-11.0.3         |       h15472ef_8       951.9 MB  conda-forge
    fftw-3.3.9                 |nompi_h74d3f13_101         6.4 MB  conda-forge
    ocl-icd-2.2.14             |       h7f98852_0         118 KB  conda-forge
    ocl-icd-system-1.0.0       |                1           4 KB  conda-forge
    openmm-7.5.0               |   py37h01de88b_6        10.7 MB  conda-forge
    ------------------------------------------------------------
                                           Total:       969.2 MB

The following NEW packages will be INSTALLED:

  cudatoolkit        conda-forge/linux-64::cudatoolkit-11.0.

In [4]:
import sys
sys.path.insert(0, "/usr/local/lib/python3.7/site-packages/")

In [5]:
!pip install --pre deepchem

     |████████████████████████████████| 552 kB 6.0 MB/s 
     |████████████████████████████████| 303 kB 8.3 MB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 27.4 MB 38 kB/s 


In [6]:
import deepchem as dc
dc.__version__

'2.5.0.dev'

In [7]:
!wget -c http://deepchem.io.s3-website-us-west-1.amazonaws.com/datasets/lipo.csv
!wget -c http://deepchem.io.s3-website-us-west-1.amazonaws.com/datasets/hppb.csv
!wget -c http://deepchem.io.s3-website-us-west-1.amazonaws.com/datasets/clearance.csv

--2021-03-05 20:18:50--  http://deepchem.io.s3-website-us-west-1.amazonaws.com/datasets/lipo.csv
Resolving deepchem.io.s3-website-us-west-1.amazonaws.com (deepchem.io.s3-website-us-west-1.amazonaws.com)... 52.219.113.67
Connecting to deepchem.io.s3-website-us-west-1.amazonaws.com (deepchem.io.s3-website-us-west-1.amazonaws.com)|52.219.113.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 282597 (276K) [text/csv]
Saving to: ‘lipo.csv’

lipo.csv            100%[===================>] 275.97K  1.35MB/s    in 0.2s    

2021-03-05 20:18:50 (1.35 MB/s) - ‘lipo.csv’ saved [282597/282597]

--2021-03-05 20:18:50--  http://deepchem.io.s3-website-us-west-1.amazonaws.com/datasets/hppb.csv
Resolving deepchem.io.s3-website-us-west-1.amazonaws.com (deepchem.io.s3-website-us-west-1.amazonaws.com)... 52.219.113.67
Connecting to deepchem.io.s3-website-us-west-1.amazonaws.com (deepchem.io.s3-website-us-west-1.amazonaws.com)|52.219.113.67|:80... connected.
HTTP request sent, await

In [8]:
import numpy as np
np.random.seed(123)
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm

import tensorflow as tf
tf.random.set_seed(123)
import deepchem as dc
from deepchem.models.graph_models import GraphConvModel

BATCH_SIZE = 128
# Set to higher values to get better numbers
MAX_EPOCH = 50
LR = 1e-3
LMBDA = 1e-4

In [9]:
def load_dataset(dataset_file, featurizer='ECFP', split='index'):
  tasks = ['target']

  if featurizer == 'ECFP':
    featurizer = dc.feat.CircularFingerprint(size=1024)
  elif featurizer == 'GraphConv':
    featurizer = dc.feat.ConvMolFeaturizer()

  loader = dc.data.CSVLoader(
      tasks=tasks, smiles_field="smile", featurizer=featurizer)
  dataset = loader.featurize(dataset_file, shard_size=8192)

  transformers = [
      dc.trans.NormalizationTransformer(transform_y=True, dataset=dataset)
  ]
  for transformer in transformers:
    dataset = transformer.transform(dataset)

  splitters = {
      'index': dc.splits.IndexSplitter(),
      'random': dc.splits.RandomSplitter(),
      'scaffold': dc.splits.ScaffoldSplitter()
  }
  splitter = splitters[split]
  train, valid, test = splitter.train_valid_test_split(dataset)
  return tasks, (train, valid, test), transformers

In [28]:
def experiment(dataset_file, method='GraphConv', split='scaffold'):
  featurizer = 'ECFP'
  if method == 'GraphConv':
    featurizer = 'GraphConv'
  tasks, datasets, transformers = load_dataset(
      dataset_file, featurizer=featurizer, split=split)
  train, val, test = datasets
  print("largo del dataset de entrenamiento", len(train))

  model = None
  if method == 'GraphConv':
    #Opciones
    #GraphConvModel(n_tasks: int, graph_conv_layers: List[int] = [64, 64], 
    #dense_layer_size: int = 128, dropout: float = 0.0, 
    #mode: str = 'classification', number_atom_features: int = 75, 
    #n_classes: int = 2, batch_size: int = 100, 
    #batch_normalize: bool = True, uncertainty: bool = False)

    #Original
    #model = GraphConvModel(len(tasks), batch_size=BATCH_SIZE, mode="regression")

    #Opcion1
    #model = GraphConvModel(len(tasks), batch_size=BATCH_SIZE, dropout=0.5, mode="regression")

    #Opcion2
    #model = GraphConvModel(len(tasks), batch_size=BATCH_SIZE, dropout=0.5, number_atom_features=64, mode="regression")

    #Opcion3
    model = GraphConvModel(len(tasks), batch_size=BATCH_SIZE, dropout=0.3, dense_layer_size = 256, mode="regression")

    #Opcion4
    #model = GraphConvModel(len(tasks), graph_conv_layers=[2048, 1024], batch_size=BATCH_SIZE, dense_layer_size = 128, mode="regression")


  elif method == 'RF':

    def model_builder_rf(model_dir):
      sklearn_model = RandomForestRegressor(n_estimators=100)
      return dc.models.SklearnModel(sklearn_model, model_dir)

    model = dc.models.SingletaskToMultitask(tasks, model_builder_rf)
  elif method == 'SVR':

    def model_builder_svr(model_dir):
      sklearn_model = svm.SVR(kernel='linear')
      return dc.models.SklearnModel(sklearn_model, model_dir)

    model = dc.models.SingletaskToMultitask(tasks, model_builder_svr)

  return model, train, val, test, transformers

In [29]:
#from keras.utils.vis_utils import plot_model
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


def benchmark(  MODEL = "GraphConv", SPLIT = "scaffold",  DATASET = "hppb.csv"):


  metric = dc.metrics.Metric(dc.metrics.pearson_r2_score, np.mean)
  
  print("About to build model")
  model, train, val, test, transformers = experiment(
      DATASET, method=MODEL, split=SPLIT)
  #plot_model(model.model,show_shapes=True, show_layer_names=True)
  
  if MODEL == 'GraphConv':
    print("running GraphConv search")
    best_val_score = 0.0
    train_score = 0.0
    best_epoch = 0
    for l in range(0, MAX_EPOCH):
      print("epoch %d" % l, end=" ")
      if (l+1) % 10 == 0:
        print()
      model.fit(train, nb_epoch=1)
      latest_train_score = model.evaluate(train, [metric],
                                          transformers)['mean-pearson_r2_score']
      latest_val_score = model.evaluate(val, [metric],
                                        transformers)['mean-pearson_r2_score']
      if latest_val_score > best_val_score:
        best_val_score = latest_val_score
        train_score = latest_train_score
        best_epoch = l
    print()
    print((MODEL, SPLIT, DATASET, train_score, best_val_score, "en la epoca:", best_epoch))
    print(model.model.summary())
  else:
    model.fit(train)
    train_score = model.evaluate(train, [metric],
                                 transformers)['mean-pearson_r2_score']
    val_score = model.evaluate(val, [metric],
                               transformers)['mean-pearson_r2_score']
    print()   
    print((MODEL, SPLIT, DATASET, train_score, val_score))

In [30]:
benchmark()

smiles_field is deprecated and will be removed in a future version of DeepChem.Use feature_field instead.


About to build model
largo del dataset de entrenamiento 1291
running GraphConv search
epoch 0 epoch 1 epoch 2 epoch 3 epoch 4 epoch 5 epoch 6 epoch 7 epoch 8 epoch 9 
epoch 10 epoch 11 epoch 12 epoch 13 epoch 14 epoch 15 epoch 16 epoch 17 epoch 18 epoch 19 
epoch 20 epoch 21 epoch 22 epoch 23 epoch 24 epoch 25 epoch 26 epoch 27 epoch 28 epoch 29 
epoch 30 epoch 31 epoch 32 epoch 33 epoch 34 epoch 35 epoch 36 epoch 37 epoch 38 epoch 39 
epoch 40 epoch 41 epoch 42 epoch 43 epoch 44 epoch 45 epoch 46 epoch 47 epoch 48 epoch 49 

('GraphConv', 'scaffold', 'hppb.csv', 0.3526140832917565, 0.1764931122208606, 'en la epoca:', 47)
Model: "private__graph_conv_keras_model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
graph_conv_14 (GraphConv)    multiple                  102144    
_________________________________________________________________
graph_conv_15 (GraphConv)    multiple                  87360   

In [ ]:
benchmark(  "GraphConv", "scaffold",  "lipo.csv")

In [ ]:
benchmark("RF", "scaffold",  "lipo.csv")

In [ ]:
benchmark(  "SVR", "scaffold",  "lipo.csv")